## 정형 데이터와 비정형 데이터
생선 - 길이, 높이, 무게 등으로 이루어진 데이터

와인 - 알코올 도수, 산도, 당도 등으로 이루어진 데이터

위와 같이 csv 파일에 보기 좋게 정리할 수 있는 데이터를 **정형 데이터(structured data)**라고 하고, 텍스트 데이터, 사진, 음악 등의 csv 파일로 정리하기 어려운 데이터를 **비정형 데이터(unstructured data)**라고 한다. 정형 데이터를 예측하는 데 가장 뛰어난 성능을 보이는 머신러닝 기법이 **앙상블 학습(ensemble learning)**이다. 앙상블 학습이란 여러 개의 약 분류기(weak classifier)를 합쳐 하나의 강 분류기(strong classifier)를 만드는 것인데, 여러 개의 결정 트리를 결합하여 하나의 결정 트리보다 더 높은 성능을 내는 기법이다. 앙상블 학습의 예시로는 랜덤 포레스트, 엑스트라 트리, 그레이디언트 부스팅, 히스토그램 기반 그레이디언트 부스팅 등이 있다

## 랜덤 포레스트
이름 그대로 여러 결정트리를 랜덤하게 만들어 각 결정 트리의 예측을 사용해 최종 예측을 만든다. '랜덤하게 만든다'에는 데이터를 랜덤하게 만든다는 의미가 포함되어있다. 예를 들어 1000개의 데이터가 있으면, 이 데이터 중에서 중복조합으로 1000개의 새로운 데이터를 만든다. 즉 1000개의 본 데이터에서 하나를 선택한 뒤 그 하나를 다시 본 데이터에 넣고, 그 다음 하나를 선택하고 다시 본 데이터에 넣는 것을 반복하여 1000개의 새로운 데이터셋을 만든다는 의미이다. 이렇게 만들어진 샘플을 **부트스트랩 샘플(bootstrap sample)**이라 부른다.

또한 각 노드를 분할할 때 전체 특성 중 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는데, 사이킷런에서 제공하는 RandomForestClassifier에서는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다. 예를 들어 4개의 특성이 있다면 무작위로 2개의 특성만 골라 분할에 사용한다

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련하여 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다. 회귀일 때는 단순하게 각 트리의 예측을 평균한다

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 불러오기
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# 타깃 데이터 분리
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()

# 훈련, 테스트셋 분리
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

# 교차 검증
# return_train_score: 검증 점수뿐만 아니라 훈련셋에 대한 점수도 함께 반환
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print()

# 특성 중요도
rf.fit(train_input, np.ravel(train_target))
print(rf.feature_importances_)

# 부트스트랩 샘플을 뽑을 때 운 나쁘게 선택되지 않은 샘플이 있는데, 이 샘플을
# OOB(out of bag) 샘플이라 하고 이 샘플을 검증셋처럼 사용할 수 있다
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, np.ravel(train_target))
print(rf.oob_score_)

0.9973541965122431 0.8905151032797809

[0.23167441 0.50039841 0.26792718]
0.8934000384837406


## 엑스트라 트리
**엑스트라 트리(Extra Trees)**는 랜덤 포레스트와 같이 100개의 결정 트리를 훈련하는데, 랜덤 포레스트와의 차이점은 부트스트랩 샘플을 사용하지 않는, 다시 말해 모든 결정 트리를 만들 때 훈련셋 전체를 사용하는 것이고, 노드를 분할할 때 최적의 분할 기준을 찾는 것이 아닌 무작위로 노드를 분할한다. 엑스트라 트리가 사용하는 결정 트리는 splitter='random'인 결정트리라 볼 수 있다

노드를 무작위로 분할해도 많은 트리를 앙상블 함으로써 오버피팅을 막고 검증셋의 점수를 높이는 효과가 있다

In [12]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


## 그레이디언트 부스팅
**그레이디언트 부스팅(gradient boosting)**은 깊이가 얕은 결정 트리를 이용해 이전 트리의 오차를 보완하는 방식으로 앙상블을 한다. 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정트리 100개를 사용한다

그레이디언트 부스팅은 랜덤 포레스트, 엑스트라 트리와는 다르게 경사 하강법을 사용하여 트리를 앙상블에 추가한다. 일반적인 경사 하강법에서는 모델의 가중치와 절편을 조금씩 바꿔가면서 손실함수의 값을 줄여나가는 방식으로 훈련했는데, 그레이디언트 부스팅은 결정 트리를 계속 추가하면서 손실 함수의 값을 줄여나간다. 이때 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 이용한다. 경사를 내려올 때는 조금씩 내려와야 하므로 깊이가 얕은 트리를 사용한다

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print()

# 학습률을 증가시키고 트리의 개수를 늘려 재학습(학습률 기본값 0.1)
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print()

# 특성 중요도 확인
gb.fit(train_input, np.ravel(train_target))
print(gb.feature_importances_)

0.8881086892152563 0.8720430147331015

0.9464595437171814 0.8780082549788999

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그레이디언트 부스팅
보통 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 낼 수 있다. 그러나 순서대로 트리를 추가하기 때문에 CPU 코어를 여러 개를 사용할 수 없고 훈련 속도가 느리다. 이런 문제점을 개선한 것이 **히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)**이며, 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘이다. 이 모델은 먼저 입력 특성을 256개의 구간으로 나누어 노드 분할 시 최적의 분할을 빠르게 찾을 수 있도록 한다. 또한 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해 사용하므로 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없어진다

사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스는 HistGradientBoostingClassifier다. 이는 트리의 개수를 지정하는데 n_estimators가 아닌 부스팅 반복 횟수를 지정하는 max_iter를 사용한다

In [15]:
# 이제 HGB는 사이킷런에서 안정화 되었으므로 experimental을 임포트 안해도 된다
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, np.ravel(train_target),
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산해보자. 이를 위해 permutation_importance()를 사용한다. 이 함수는 특성을 하나씩 무작위로 섞어 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다

n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정하며, 기본값은 5다. permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)를 담고 있다

In [16]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, np.ravel(train_target))

# 훈련셋 특성 중요도
result = permutation_importance(hgb, train_input, np.ravel(train_target),
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
print()

# 테스트셋 특성 중요도
result = permutation_importance(hgb, test_input, np.ravel(test_target),
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]

[0.05969231 0.20238462 0.049     ]


In [20]:
# 사이킷런 HGB 성능 테스트
hgb.score(test_input, np.ravel(test_target))

# XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, np.ravel(train_target),
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
print()

# 마이크로소프트의 LightGBM(사이킷런의 HGB에 많은 영향)
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, np.ravel(train_target),
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284

0.9338079582727165 0.8789710890649293
